In [10]:
!pip install -q langchain langchain-google-genai "google-ai-generativelanguage>=0.6.18,<0.7.0"

In [ ]:
import os
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict, List, Dict


In [30]:
# 환경 변수에서 키 불러오기
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID", "")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET", "")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)
parser = StrOutputParser()

# 🟠 1단계: 캠핑 준비물 생성
def generate_checklist(state):
    user_input = state["question"]
    prompt = f"""
    사용자가 '{user_input}' 라고 했어. 캠핑에 필요한 준비물 목록을 5~7개 정도 추천해줘.
    단순 나열 형식으로 출력해줘 (예: 텐트, 침낭, 휴대용 가스버너, ...)"""
    checklist = llm.invoke([HumanMessage(content=prompt)])
    print(f"\n📝 [generate_checklist] question: {state['question']}")
    return {"checklist": checklist.content.strip()}

In [26]:

# 🟠 2단계: 각 준비물에 대해 SEO 최적 키워드 생성
def generate_keywords(state):
    checklist = state["checklist"].split(", ")
    keyword_prompt = f"""
    다음 캠핑 준비물 리스트가 있어: {', '.join(checklist)}.
    각각에 대해 네이버 쇼핑 검색에 적합한 SEO 최적 키워드 (상세하고 일반적으로 검색되는 형태)로 바꿔줘.
    예시 형식:
    텐트 → 4인용 자동 캠핑 텐트
    침낭 → 겨울용 거위털 침낭
    형식 유지해서 출력해줘.
    """
    keyword_response = llm.invoke([HumanMessage(content=keyword_prompt)])
    lines = keyword_response.content.strip().split("\n")
    keywords = [line.split("→")[1].strip() for line in lines if "→" in line]
    return {"keywords": keywords}

In [27]:

# 🟠 3단계: 네이버 API로 검색
def search_naver_items(state):
    results = []
    for keyword in state["keywords"]:
        url = "https://openapi.naver.com/v1/search/shop.json"
        headers = {
            "X-Naver-Client-Id": NAVER_CLIENT_ID,
            "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
        }
        params = {"query": keyword, "display": 3, "start": 1, "sort": "sim"}
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            items = response.json().get("items", [])
            results.append({
                "keyword": keyword,
                "items": items
            })
    return {"search_results": results}


In [28]:

# 🟠 4단계: 상품 비교 및 추천 (LLM 활용)
def compare_and_recommend(state):
    recommendation = []
    for group in state["search_results"]:
        keyword = group["keyword"]
        items = group["items"]
        summary_prompt = f"""
        다음은 '{keyword}'로 검색한 네이버 쇼핑 상품 3개야:

        {items[0]['title']} - {items[0]['lprice']}원
        {items[1]['title']} - {items[1]['lprice']}원
        {items[2]['title']} - {items[2]['lprice']}원

        각각의 장단점을 간단히 비교해주고, 그 중 가장 추천하는 하나의 제품을 골라줘.
        """
        summary = llm.invoke([HumanMessage(content=summary_prompt)])
        recommendation.append({
            "keyword": keyword,
            "summary": summary.content.strip()
        })
    return {"recommendations": recommendation}

In [ ]:
# 상태 스키마 정의
class AppState(TypedDict):
    question: str
    checklist: str
    keywords: List[str]
    search_results: List[Dict]
    recommendations: List[Dict]

# ✅ 올바른 LangGraph 구성
builder = StateGraph(state_schema=AppState)
builder.add_node("ChecklistGenerator", RunnableLambda(generate_checklist))
builder.add_node("KeywordGenerator", RunnableLambda(generate_keywords))
builder.add_node("NaverSearcher", RunnableLambda(search_naver_items))
builder.add_node("Recommender", RunnableLambda(compare_and_recommend))

builder.set_entry_point("ChecklistGenerator")
builder.add_edge("ChecklistGenerator", "KeywordGenerator")
builder.add_edge("KeywordGenerator", "NaverSearcher")
builder.add_edge("NaverSearcher", "Recommender")
builder.add_edge("Recommender", END)

app = builder.compile()

# 실행
output = app.invoke({"question": "나는 내일 캠핑갈건데 뭐가 필요할까?"})
for rec in output["recommendations"]:
    print(f"\n📌 [{rec['keyword']}] 추천 요약:\n{rec['summary']}")


IndexError: list index out of range

In [2]:
# -*- coding: utf-8 -*-
# !pip install -q langchain langchain-google-genai "google-ai-generativelanguage>=0.6.18,<0.7.0"

import os
import re
import requests
from typing import TypedDict, List, Dict
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage

# 🔐 환경 변수 설정 또는 기본값
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID", "")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET", "")

# 🔗 LLM 초기화
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

# 🟠 1단계: 상품 리스트 생성 (노드 1)
def generate_checklist(state):
    user_input = state["question"]
    prompt = f"""
    다음은 사용자의 상황입니다: '{user_input}'

    사용자의 말에 자연스럽게 반응해주고, 어떤 물건을 추천하는지 간단히 설명해줘.
    그 뒤에 [추천 리스트: 항목1, 항목2, 항목3, ...] 형식으로 추천 아이템을 따로 명시해줘.

    예시:
    - 사용자가 '내일 캠핑갈건데 뭐가 필요할까?'라고 하면 →  
    캠핑 가신다니 설레네요! 텐트, 침낭, 매트는 꼭 챙기시면 좋아요.  
    [추천 리스트: 텐트, 침낭, 매트]

    - 사용자가 '물놀이 가게 됐어'라고 하면 →  
    재밌겠네요! 수영복, 물안경, 방수팩은 필수예요.  
    [추천 리스트: 수영복, 물안경, 방수팩]

    지금 사용자 입력: '{user_input}'
    """
    full_response = llm.invoke([HumanMessage(content=prompt)]).content.strip()

    # 리스트 추출
    match = re.search(r"\[추천 리스트: (.*?)\]", full_response)
    checklist_str = match.group(1) if match else ""
    checklist = [item.strip() for item in checklist_str.split(",") if item.strip()]

    print("\n📝 [ChecklistGenerator]")
    print("📥 입력 질문:", user_input)
    print("📤 전체 응답:", full_response)
    print("📤 추출된 리스트:", checklist)

    return {
        "question": user_input,
        "checklist": ", ".join(checklist),
        "checklist_message": full_response
    }

# 🟠 2단계: SEO 최적 키워드 생성 (노드 2) + 카테고리화
def generate_keywords(state):
    checklist = state["checklist"].split(", ")
    user_input = state["question"]

    prompt = f"""
    사용자의 질문: '{user_input}'
    추천된 준비물 리스트: {', '.join(checklist)}

    각 항목에 대해:
    1. 사람들이 자주 검색할 법한 SEO 키워드로 변환하고
    2. 첨부된 네이버 쇼핑 카테고리 중 하나에 분류해줘.
       카테고리 목록:
       여성의류, 남성의류, 패션잡화, 신발, 화장품/미용, 신선식품, 가공식품, 건강식품,
       출산/유아동, 반려동물용품, 가전, 휴대폰/카메라, PC/주변기기, 가구,
       조명/인테리어, 패브릭/홈데코, 주방용품, 생활용품, 스포츠/레저, 자동차/오토바이,
       키덜트/취미, 건강의료용품, 악기/문구, 공구, 렌탈관, e쿠폰/티켓/생활편의, 여행

    형식:
    항목 → SEO 키워드 | 카테고리

    지금 변환할 리스트:
    {', '.join(checklist)}
    """

    response = llm.invoke([HumanMessage(content=prompt)]).content.strip()
    lines = response.split("\n")

    keywords = []
    for line in lines:
        if "→" in line and "|" in line:
            try:
                _, pair = line.split("→")
                keyword, category = pair.split("|")
                keywords.append({
                    "keyword": keyword.strip(),
                    "category": category.strip()
                })
            except:
                continue

    print("\n🔑 [KeywordGenerator]")
    print("📥 입력 체크리스트:", checklist)
    print("📤 추출된 키워드+카테고리:", keywords)

    return {"checklist": state["checklist"], "keywords": keywords}

    # 예시 형식:
    # 텐트 → 4인용 자동 캠핑 텐트
    # 침낭 → 겨울용 거위털 침낭
    # 무좀 약 → 효과 좋은 무좀 치료제
    # 수영복 → 여성용 래쉬가드 수영복

# 🟠 3단계: 네이버 쇼핑 API 검색
def search_naver_items(state):
    results = []
    for keyword in state["keywords"]:
        url = "https://openapi.naver.com/v1/search/shop.json"
        headers = {
            "X-Naver-Client-Id": NAVER_CLIENT_ID,
            "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
        }
        params = {"query": keyword, "display": 3, "start": 1, "sort": "sim"}
        response = requests.get(url, headers=headers, params=params)
        items = response.json().get("items", []) if response.status_code == 200 else []

        print(f"\n🔍 [NaverSearcher] 키워드: '{keyword}'")
        print(f"🔎 검색된 상품 수: {len(items)}")
        print(f"🔎 상품 JSON 예시:")
        for item in items:
            print(item)

        results.append({"keyword": keyword, "items": items})

    return {"keywords": state["keywords"], "search_results": results}

# 🟠 4단계: 상품 비교 및 추천 물품별 {title, 가격}
def compare_and_recommend(state):
    CATEGORY_PROMPT_MAP = {
        "여성의류": "디자인, 계절 적합성, 소재, 사이즈 다양성, 가격 등을 기준으로 비교해줘.",
        "남성의류": "스타일, 활용도, 소재 품질, 브랜드, 가격 등을 기준으로 비교해줘.",
        "패션잡화": "디자인, 실용성, 브랜드, 재질, 가격 등을 기준으로 비교해줘.",
        "신발": "착화감, 용도, 재질, 내구성, 디자인, 가격 등을 기준으로 비교해줘.",
        "화장품/미용": "성분, 피부타입 적합성, 사용감, 브랜드, 가격 등을 기준으로 비교해줘.",
        "신선식품": "신선도, 영양 성분, 수확 시기, 원산지, 유통기한 등을 기준으로 비교해줘.",
        "가공식품": "맛, 유통기한, 성분, 조리 편의성, 가격 등을 기준으로 비교해줘.",
        "건강식품": "주요 영양소, 건강에 미치는 효과, 복용 편의성, 인증 여부, 가격 등을 비교해줘.",
        "출산/유아동": "안전성, 피부 친화성, 연령 적합성, 기능성, 가격 등을 기준으로 비교해줘.",
        "반려동물용품": "안전성, 반려동물 선호도, 기능, 재질, 가격 등을 기준으로 비교해줘.",
        "가전": "성능, 에너지 효율, 브랜드 신뢰도, 편의 기능, 가격 등을 기준으로 비교해줘.",
        "휴대폰/카메라": "성능, 브랜드, 배터리 수명, 기능, 가격 등을 기준으로 비교해줘.",
        "PC/주변기기": "성능, 호환성, 브랜드, 기능성, 가격 등을 기준으로 비교해줘.",
        "가구": "디자인, 내구성, 크기, 수납 기능, 가격 등을 기준으로 비교해줘.",
        "조명/인테리어": "밝기, 디자인, 에너지 효율, 설치 편의성, 가격 등을 기준으로 비교해줘.",
        "패브릭/홈데코": "재질, 세탁 용이성, 디자인, 계절성, 가격 등을 기준으로 비교해줘.",
        "주방용품": "재질, 내구성, 세척 용이성, 기능성, 가격 등을 기준으로 비교해줘.",
        "생활용품": "실용성, 내구성, 사용 편의성, 디자인, 가격 등을 기준으로 비교해줘.",
        "스포츠/레저": "내구성, 사용 목적 적합성, 기능성, 휴대성, 가격 등을 기준으로 비교해줘.",
        "자동차/오토바이": "성능, 브랜드, 연비, 안전 기능, 유지비용 등을 기준으로 비교해줘.",
        "키덜트/취미": "희소성, 디자인, 만족도, 수집 가치, 가격 등을 기준으로 비교해줘.",
        "건강의료용품": "정확성, 안전성, 사용 편의성, 인증 여부, 가격 등을 기준으로 비교해줘.",
        "악기/문구": "사용감, 내구성, 기능성, 브랜드, 가격 등을 기준으로 비교해줘.",
        "공구": "내구성, 사용 용이성, 기능, 브랜드, 가격 등을 기준으로 비교해줘.",
        "렌탈관": "렌탈 기간, 비용, 유지보수 조건, 최신 모델 여부, 브랜드 신뢰도 등을 기준으로 비교해줘.",
        "e쿠폰/티켓/생활편의": "사용처, 유효기간, 할인율, 사용 조건, 가격 등을 기준으로 비교해줘.",
        "여행": "여행지 매력도, 일정 구성, 가격, 포함 혜택, 후기 평점 등을 기준으로 비교해줘.",
    }   

    recommendations = []
    for idx, group in enumerate(state["search_results"]):
        keyword_info = state["keywords"][idx]
        keyword = keyword_info["keyword"]
        category = keyword_info["category"]
        items = group["items"]

        print(f"\n🤖 [Recommender] 키워드: '{keyword}' | 카테고리: {category}")
        print(f"🛒 상품 수: {len(items)}")

        if len(items) < 3:
            summary = f"'{keyword}'에 대한 상품이 부족하여 비교할 수 없습니다."
        else:
            compare_criteria = CATEGORY_PROMPT_MAP.get(
                category, "가격, 성능, 사용자 리뷰, 가성비 등을 기준으로 비교해줘."
            )
            summary_prompt = f"""
            다음은 네이버 쇼핑에서 '{keyword}' 키워드로 검색한 상품 3개입니다:

            1. {items[0]['title']} - {items[0]['lprice']}원  
            2. {items[1]['title']} - {items[1]['lprice']}원  
            3. {items[2]['title']} - {items[2]['lprice']}원  

            {compare_criteria}
            그리고 가장 추천할 제품 하나를 골라 이유와 함께 말해줘.
            """
            
            summary = llm.invoke([HumanMessage(content=summary_prompt)]).content.strip()

        recommendations.append({
            "keyword": keyword,
            "category": category,
            "summary": summary
        })

    return {"search_results": state["search_results"], "recommendations": recommendations}

# 💡 상태 스키마 정의
class AppState(TypedDict):
    question: str
    checklist_message: str  # 사용자 응답용 자연어 문장
    checklist: str
    keywords: List[Dict[str, str]]  # 각 dict는 {"keyword": SEO 키워드, "category": 카테고리}
    search_results: List[Dict]
    recommendations: List[Dict]

# 🧠 LangGraph 구성
builder = StateGraph(state_schema=AppState)
builder.add_node("ChecklistGenerator", RunnableLambda(generate_checklist))
builder.add_node("KeywordGenerator", RunnableLambda(generate_keywords))
builder.add_node("NaverSearcher", RunnableLambda(search_naver_items))
builder.add_node("Recommender", RunnableLambda(compare_and_recommend))

builder.set_entry_point("ChecklistGenerator")
builder.add_edge("ChecklistGenerator", "KeywordGenerator")
builder.add_edge("KeywordGenerator", "NaverSearcher")
builder.add_edge("NaverSearcher", "Recommender")
builder.add_edge("Recommender", END)

app = builder.compile()

# 🚀 실행
# output = app.invoke({"question": "나는 내일 캠핑갈건데 뭐가 필요할까?"})
output = app.invoke({"question": "아무 상품이나 추천해줘."})

# ✅ 최종 결과 출력
print("\n🎯 최종 추천 요약 결과")
for rec in output["recommendations"]:
    print(f"\n📌 [{rec['keyword']}] 추천 요약:\n{rec['summary']}")


📝 [ChecklistGenerator]
📥 입력 질문: 아무 상품이나 추천해줘.
📤 전체 응답: 음, 그럼 오늘 날씨에 딱 맞는 시원한 아이템을 추천해 드릴게요!  요즘 같은 날씨에는 시원한 음료와 함께 휴식을 취하는 게 최고잖아요.  얼음 가득한 컵에 시원한 음료를 마시며 편안한 시간을 보내보세요!  

[추천 리스트: 얼음 트레이, 텀블러, 아이스커피(혹은 좋아하는 음료)]
📤 추출된 리스트: ['얼음 트레이', '텀블러', '아이스커피(혹은 좋아하는 음료)']

🔑 [KeywordGenerator]
📥 입력 체크리스트: ['얼음 트레이', '텀블러', '아이스커피(혹은 좋아하는 음료)']
📤 추출된 키워드+카테고리: [{'keyword': '얼음틀, 실리콘 얼음틀, 아이스 트레이, 대용량 얼음틀, 얼음 제빙기', 'category': '주방용품'}, {'keyword': '보냉 텀블러, 스테인리스 텀블러, 텀블러 추천, 대용량 텀블러, 개인 텀블러, 보온 텀블러', 'category': '생활용품'}, {'keyword': '아이스커피, 냉커피, 여름 음료, 시원한 음료,  커피믹스,  수제청,  탄산음료', 'category': '가공식품 (아이스커피의 경우),  신선식품(수제청의 경우)  *음료 종류에 따라 카테고리가 달라짐.  좋아하는 음료를 명시해야 더 정확한 분류가 가능합니다.'}]

🔍 [NaverSearcher] 키워드: '{'keyword': '얼음틀, 실리콘 얼음틀, 아이스 트레이, 대용량 얼음틀, 얼음 제빙기', 'category': '주방용품'}'
🔎 검색된 상품 수: 3
🔎 상품 JSON 예시:
{'title': '빅 브라이트 이지 시 키보드 및 마우스 - USB 유선 고대비 옐로우 오버사이즈 블랙 문자 포함 시니어 시각 장애인용 쿼티 대형 인쇄 키(키보드 마우스)  <b>Keyword</b> Only', 'link': 'https://link.coupang.com/re/PCSNAVERPCSDP?pageKey=8706678197&